In [107]:
/*
 * Si Python et/ou R ne fonctionnent pas, voyez: https://github.com/dotnet/interactive/blob/HEAD/docs/jupyter-in-polyglot-notebooks.md
 *
 * Enlevez les commentaires qui suivent afin de connecter Python et R.
 */

// #!connect jupyter --kernel-name pythonkernel --kernel-spec python3

// #!connect jupyter --kernel-name Rkernel --kernel-spec ir

In [108]:
#r "nuget: Microsoft.Extensions.Http, 8.0.0"

#r "nuget: GBX.NET, 2.0.9"
#r "nuget: GBX.NET.LZO, 2.1.1"
#r "nuget: GBX.NET.ZLib, 1.1.0"
#r "nuget: CsvHelper, 33.0.1"

Installed Packages CsvHelper, 33.0.1 GBX.NET, 2.0.9 GBX.NET.LZO, 2.1.1 GBX.NET.ZLib, 1.1.0 Microsoft.Extensions.Http, 8.0.0

## Assigner les valeurs d'échantillonage

Modifiez les valeurs d'échantillonage afin de modifier la taille, le nombre de records sur chaque carte, et cetera.

In [109]:
using System.IO;

public class SampleSettings
{
  public static int minRecords = 5;

  public static int sampleSizePerTag = 1;

  public static int softMaximumAttemptsPerQuery = 20;
  public static int hardMaximumAttemptsPerQuery = 100;
}

public class GeneralSettings {
  public static string dataFolderName = @"previous-data";

  public static string individualDataFolderName = GenerateDataFolderName();

  public static string replayDataFileName = @"replay-data.json";

  public static string flatReplayDataFileName = @"flat-replay-data.csv";

  public static string combinedFlatReplayDataFileName = SampleSettings.minRecords + "recs-combined-flat-data.csv";

  public static string combinedReplayDataFileName = SampleSettings.minRecords + "recs-combined-data.json";

  public static string GenerateDataFolderName() {
    for (int i = 1; i < int.MaxValue; i++) {
      string folderName = "./" + dataFolderName + "/" + SampleSettings.sampleSizePerTag + "per-" + SampleSettings.minRecords + "rep-" + i + "/";
      if (!Directory.Exists(folderName)) {
        return folderName;
      }
    }
    
    throw new OverflowException("No directory could be created");
  }

  public static string AppendIndividualDataFolderName(string fileName) {
    if (!Directory.Exists(individualDataFolderName)) {
      Directory.CreateDirectory(individualDataFolderName);
    }

    return individualDataFolderName + fileName;
  }
}

## Préparation pour l'utilisation du API de TMNF-X (Trackmania Nations Forever Exchange), ansi que de leur site normal

In [110]:

public enum Order1 {
  None = 0,
  UploadDateOldest = 1,
  UploadDateNewestDefault = 2,
  UpdateDateOldest = 3,
  UpdateDateNewest = 4,
  AwardCountAscending = 5,
  AwardCountDescending = 6,
  CommentCountAscending = 7,
  CommentCountDescending = 8,
  ActivityOldest = 9,
  ActivityNewest = 10,
  TrackNameAZ = 11,
  TrackNameZA = 12,
  AuthorNameAZ = 13,
  AuthorNameZA = 14,
  DifficultyAscending = 15,
  DifficultyDescending = 16,
  DownloadCountAscending = 17,
  DownloadCountDescending = 18,
  TrackValueAscending = 19,
  TrackValueDescending = 20,
  AwardsthisweekAscending = 21,
  AwardsthisweekDescending = 22,
  AwardsthismonthAscending = 23,
  AwardsthismonthDescending = 24,
  LastAwardDateOldest = 25,
  LastAwardDateNewest = 26,
  WorldRecordsetOldest = 27,
  WorldRecordsetNewest = 28,
  PersonalrecordsetOldestLoginrequired = 29,
  PersonalrecordsetNewestLoginrequired = 30,
  PersonalawardsetOldestLoginrequired = 31,
  PersonalawardsetNewestLoginrequired = 32,
  PersonaldownloadOldestLoginrequired = 33,
  PersonaldownloadNewestLoginrequired = 34,
  PersonalcommentsetOldestLoginrequired = 35,
  PersonalcommentsetNewestLoginrequired = 36,
  UserRecordPositionLowest = 37,
  UserRecordPositionHighest = 38,
}

public enum Tag {
  Normal = 0,
  Stunt = 1,
  Maze = 2,
  Offroad = 3,
  Laps = 4,
  Fullspeed = 5,
  LOL = 6,
  Tech = 7,
  SpeedTech = 8,
  RPG = 9,
  PressForward = 10,
  Trial = 11,
  Grass = 12,
}

public enum UsableTag {
  Normal = 0,
  //Stunt = 1,
  //Maze = 2,
  Offroad = 3,
  //Laps = 4,
  Fullspeed = 5,
  LOL = 6,
  Tech = 7,
  SpeedTech = 8,
  //RPG = 9,
  PressForward = 10,
  //Trial = 11,
  Grass = 12,
}

public enum PrimaryType {
  Race = 0,
  Puzzle = 1,
  Platform = 2,
  Stunts = 3,
  Shortcut = 4,
  Laps = 5,
}

public enum LBType {
  Allreplaytypes = -1,
  Normal = 0,
  Classic = 1,
  Nadeo = 2,
  Useless = 3,
}

public enum Difficulty {
  Beginner = 0,
  Intermediate = 1,
  Expert = 2,
  Lunatic = 3,
}

public enum Routes {
  Single = 0,
  Multiple = 1,
  Symmetrical = 2,
}

In [111]:
#nullable enable

public class TMXResponses
{
  public static DateTime newestMapAt { get; set; }
  public static DateTime oldestMapAt { get; set; }

  public class ParsedResponse<T>
  {
    public Boolean More { get; set; }
    public T[]? Results { get; set; }
  }

  public class MapResponse
  {
      public int? TrackId { get; set; }
      public string? TrackName { get; set; }
      public string? UId { get; set; }
      public int? AuthorTime { get; set; }
      public Uploader? Uploader { get; set; }
      public DateTime? UploadedAt { get; set; }
      public DateTime? UpdatedAt { get; set; }
      public int? PrimaryType { get; set; } // enum
      public int? TrackValue { get; set; }
      public int? Style { get; set; } // enum
      public int? Routes { get; set; } // enum
      public int? Difficulty { get; set; }
      public int? Environment { get; set; } // enum
      public int? Car { get; set; } // enum
      public WRReplay? WRReplay { get; set; }
      public int[]? Tags { get; set; }
  }

  public class Uploader
  {
    public int? UserId { get; set; }
    public string? Name { get; set; }
  }

  public class WRReplay
  {
    public int ReplayId { get; set; }
    public int ReplayTime { get; set; }
  }

  public class ReplayResponse
  {
      public Int64? ReplayId { get; set; }
      public User? User { get; set; }
      public Int32? ReplayTime { get; set; }
      public Int32? ReplayScore { get; set; }
      public Int32? ReplayRespawns { get; set; }
      public Int32? Score { get; set; }
      public Int32? Position { get; set; }
      public Int32? IsBest { get; set; }
      public Int32? IsLeaderboard { get; set; }
      public DateTime? TrackAt { get; set; }
      public DateTime? ReplayAt { get; set; }
  }

  public class User
  {
    public int? UserId { get; set; }
    public String? Name { get; set; }
  }
}

In [112]:
#nullable enable

public class TMXInputs
{
  private static String inputParametersToString<T>(T inputParametersObject)
  {
    // Voir https://stackoverflow.com/questions/4276566/how-can-you-loop-over-the-properties-of-a-class

    StringBuilder paramaterString = new StringBuilder();

    Boolean needsAnd = false;
    foreach(var prop in typeof(T).GetProperties()) {
      object? value = prop.GetValue(inputParametersObject, null);

      if (value == null) { continue; }

      if (needsAnd) {
        paramaterString.Append("&");
      } else { needsAnd = true; }

      if (prop.Name == "fields") {
        paramaterString.Append($"fields={String.Join("%2C", (String[]) value)}");
        continue;
      }

      // If DateTime
      if (value is DateTime) {
        string dateString = ((DateTime) value).ToString("s", System.Globalization.CultureInfo.InvariantCulture);

        paramaterString.Append($"{prop.Name}={dateString}");
        continue;
      }

      // If Enum[]
      if (value.GetType().IsArray && (((Array) value).GetValue(0)??"NotEnum").GetType().IsEnum) {
        paramaterString.Append($"{prop.Name}=");

        int[] enumValue =  (int[]) value;

        paramaterString.Append(enumValue[0]);

        for (int i = 1; i < enumValue.Length; i++) {
          paramaterString.Append($"%2C{enumValue[i]}");
        }
        
        continue;
      }

      // If Enum
      if (value is Enum) {
        paramaterString.Append($"{prop.Name}={((Enum) value).ToString("D")}");
        continue;
      }

      paramaterString.Append($"{prop.Name}={value}");
    }
    
    return paramaterString.ToString();
  }

  public class MapInputsParameters {
      /// <summary>
      /// Comma-seperated and URL-Encoded list of fields to display in the response.
      /// </summary>
    public String[]? fields { get; set; }
      /// <summary>
      /// Search Order
      /// </summary>
    public Order1? order1 { get; set; }
      /// <summary>
      /// Amount of results (default: 40)
      /// </summary>
    public Int32? count { get; set; }
      /// <summary>
      /// Display results after specified TrackId and given order - used for paging
      /// </summary>
    public Int64? after { get; set; }
      /// <summary>
      /// Display results before specified TrackId and given order - used for paging
      /// </summary>
    public Int64? before { get; set; }
      /// <summary>
      /// Like after, but includes the specified TrackId as well
      /// </summary>
    public Int64? from { get; set; }
      /// <summary>
      /// Filter by TrackId's
      /// </summary>
    public Int64[]? id { get; set; }
      /// <summary>
      /// Filter by UId or Track file signatures
      /// </summary>
    public String[,]? uid { get; set; }
      /// <summary>
      /// Filter by Tags[]
      /// </summary>
    public Tag[]? tag { get; set; }
      /// <summary>
      /// Filter by excluding Tags[]
      /// </summary>
    public Tag[]? etag { get; set; }
      /// <summary>
      /// true: track must have all specified tags, false: track must have one of the specified tags
      /// </summary>
    public Boolean taginclusive { get; } = true;
      /// <summary>
      /// Filter by PrimaryType
      /// </summary>
    public PrimaryType primarytype { get; } = PrimaryType.Race;
      /// <summary>
      /// Filter by min. UploadedAt
      /// </summary>
    public DateTime? uploadedafter { get; set; }
      /// <summary>
      /// Filter by max. UploadedAt
      /// </summary>
    public DateTime? uploadedbefore { get; set; }
      /// <summary>
      /// Filter by min. AuthorTime in ms
      /// </summary>
    public Int32? authortimemin { get; } = 30000;
      /// <summary>
      /// Filter by max. AuthorTime in ms
      /// </summary>
    public Int32? authortimemax { get; } = 120000;
      /// <summary>
      /// Filter by Routes
      /// </summary>
    public Routes routes { get; } = Routes.Single;
      /// <summary>
      /// 1: Track has at least one replay
      /// </summary>
    public Int32 inhasrecord { get; } = 1;
      /// <summary>
      /// 1: Track is environment mixed (Environment =/= Car)
      /// </summary>
    public Int32 inenvmix { get; } = 0;
      /// <summary>
      /// 1: Track requires to be played with TMUnlimiter, 0: vanilla game
      /// </summary>
    public Int32 inunlimiter { get; } = 0;

    public override string ToString(){
      return inputParametersToString<MapInputsParameters>(this);
    }  
  }

  public class ReplayInputsParameters {
      /// <summary>
      /// TM1X TrackId of track containing the replays
      /// </summary>
    public String? trackId { get; set; }
      /// <summary>
      /// Comma-seperated and URL-Encoded list of fields to display in the response.
      /// </summary>
    public String[]? fields { get; set; }
      /// <summary>
      /// Amount of results (default: 10)
      /// </summary>
    public Int32? count { get; set; }
      /// <summary>
      /// Display results after specified ReplayId - used for paging
      /// </summary>
    public Int64? after { get; set; }
      /// <summary>
      /// Display results before specified ReplayId - used for paging
      /// </summary>
    public Int64? before { get; set; }
      /// <summary>
      /// Like after, but includes the specified ReplayId as well
      /// </summary>
    public Int64? from { get; set; }
      /// <summary>
      /// 1: Returns only the best replay
      /// </summary>
    public Int32? best { get; set; } = 1;
      /// <summary>
      /// Filter by driver TM1X UserId
      /// </summary>
    public Int64? userid { get; set; }

    public override string ToString(){
      return inputParametersToString<ReplayInputsParameters>(this);
    }  
  }
}

In [113]:
using GBX.NET;
using GBX.NET.Engines.Game;
using GBX.NET.Engines.Plug;

#nullable enable

public class DataContainers {
  public class Vector3 {
    public float X { get; set; } = 0;
    public float Y { get; set; } = 0;
    public float Z { get; set; } = 0;
  }

  public class SampleData {
      /// <summary>
      /// Temps actuel de l'échantillon, ayant comme unité 10 millisecondes
      /// </summary>
    public float? Time { get; set; }

      /// <summary>
      /// Révolutions par minute
      /// </summary>
    public float? RPM { get; set; }

      /// <summary>
      /// Le pilotage de la voiture, soit au travers du volan en-jeu.
      /// Un pilotage positif tourne vers la droite et un pilotage négatif tourne vers la gauche.
      /// </summary>
    public float? Steer { get; set; }

      /// <summary>
      /// Vitesse vers l'avant de la voiture
      /// </summary>
    public float? SpeedForward { get; set; }
      /// <summary>
      /// Vitesse vers le coté de la voiture, soit perpendiculaire à la direction dont la voiture fait face.
      /// La vitesse pointe vers la gauche lorsque la vitesse est positive et vers la droite lorsque la vitesse est négative.
      /// </summary>
    public float? SpeedSideward { get; set; }

      /// <summary>
      /// Vecteur-vitesse angulaire en 3 dimensions
      /// </summary>
    public Vector3? AngularVelocity { get; set; } = new Vector3();

      /// <summary>
      /// Tangage de la voiture
      /// </summary>
    public Double? Pitch { get; set; }
      /// <summary>
      /// Lacet de la voiture
      /// </summary>
    public Double? Yaw { get; set; }
      /// <summary>
      /// Roulis de la voiture
      /// </summary>
    public Double? Roll { get; set; }

      /// <summary>
      /// Position de la voiture à un instant en 3 dimensions
      /// </summary>
    public Vector3? Position { get; set; } = new Vector3();

      /// <summary>
      /// Booléen de la présence d'un turbo
      /// </summary>
    public Boolean? IsTurbo { get; set; }
    
    ///////////////////////////////////////////////////////////// Non nécessaire
    //   /// <summary>
    //   /// Matériel de contact de la roue avant-droite
    //   /// </summary>
    // public CPlugSurface.MaterialId? FRGroundContactMaterial { get; set; }
    //   /// <summary>
    //   /// Matériel de contact de la roue avant-gauche
    //   /// </summary>
    // public CPlugSurface.MaterialId? FLGroundContactMaterial { get; set; }
    //   /// <summary>
    //   /// Matériel de contact de la roue arrière-droite
    //   /// </summary>
    // public CPlugSurface.MaterialId? RRGroundContactMaterial { get; set; }
    //   /// <summary>
    //   /// Matériel de contact de la roue arrière-gauche
    //   /// </summary>
    // public CPlugSurface.MaterialId? RLGroundContactMaterial { get; set; }

    //   /// <summary>
    //   /// La longueur d'ammortissement de la roue avant-droite
    //   /// </summary>
    // public float? FRDampenLen { get; set; }
    //   /// <summary>
    //   /// La longueur d'ammortissement de la roue avant-gauche
    //   /// </summary>
    // public float? FLDampenLen { get; set; }
    //   /// <summary>
    //   /// La longueur d'ammortissement de la roue arrière-droite
    //   /// </summary>
    // public float? RRDampenLen { get; set; }
    //   /// <summary>
    //   /// La longueur d'ammortissement de la roue arrière-gauche
    //   /// </summary>
    // public float? RLDampenLen { get; set; }
  }

  public class ReplayData {
      /// <summary>
      /// L'étiquette du circuit.
      /// </summary>
    public String? Tag { get; set; }
      /// <summary>
      /// Durée totale du replay, ayant comme unité 10 millisecondes.
      /// </summary>
    public float? Duration { get; set; }
      /// <summary>
      /// Les échantillons du replay
      /// </summary>
    public List<SampleData>? Samples { get; set; }
  }

  public class FlatData {
      /// <summary>
      /// L'étiquette du circuit.
      /// </summary>
    public string? Tag { get; set; }

      /// <summary>
      /// Moyenne absolue de différences de positions pour l'hypothénuse des axes des X et Z
      /// </summary>
    public double? AvgAbsDisplacementHorizontal { get; set; }
      /// <summary>
      /// Moyenne absolue de différences de positions pour l'axe des Y
      /// </summary>
    public double? AvgAbsDisplacementY { get; set; }

      /// <summary>
      /// Moyenne absolue des rotations par minute
      /// </summary>
    public double? AvgRPM { get; set; }

      /// <summary>
      /// Moyenne des pilotages
      /// </summary>
    public double? AvgSteerBias { get; set; }
      /// <summary>
      /// Moyenne absolue des pilotages
      /// </summary>
    public double? AvgAbsSteer { get; set; }

      /// <summary>
      /// Moyenne des vitesses vers l'avant
      /// </summary>
    public double? AvgSpeedForward { get; set; }
      /// <summary>
      /// Moyenne absolue des vitesses vers l'avant
      /// </summary>
    public double? AvgAbsSpeedForward { get; set; }

      /// <summary>
      /// Moyenne des vitesses latérales
      /// </summary>
    public double? AvgSpeedSidewardBias { get; set; }
      /// <summary>
      /// Moyenne absolue des vitesses latérales
      /// </summary>
    public double? AvgAbsSpeedSideward { get; set; }

      /// <summary>
      /// Un nombre de -1 à 1 indiquant l'inclinaison moyenne de l'opposition des directions de la vitesse latérale et du pilotage.
      /// Positif s'ils sont du même signe, n'égatif s'ils sont de signes différents
      /// </summary>
    public double? AvgSpeedSidewardOppSteer { get; set; }

      /// <summary>
      /// Le pourcentage des échantillons où le tnagage absolu se trouve plus bas que le tiers de sa valeur maximale
      /// </summary>
    public double? PercentPitchLowerThird { get; set; }
      /// <summary>
      /// Le pourcentage des échantillons où le tangage absolu se trouve entre le tiers et deux tiers de sa valeur maximale
      /// </summary>
    public double? PercentPitchMiddleThird { get; set; }
      /// <summary>
      /// Le pourcentage des échantillons où le tangage absolu se trouve plus haut que les deux tiers de la valeur maximale
      /// </summary>
    public double? PercentPitchUpperThird { get; set; }

      /// <summary>
      /// Le pourcentage des échantillons où le roulis absolu se trouve plus bas que le tiers de sa valeur maximale
      /// </summary>
    public double? PercentRollLowerThird { get; set; }
      /// <summary>
      /// Le pourcentage des échantillons où le roulis absolu se trouve entre le tiers et deux tiers de sa valeur maximale
      /// </summary>
    public double? PercentRollMiddleThird { get; set; }
      /// <summary>
      /// Le pourcentage des échantillons où le roulis absolu se trouve plus haut que les deux tiers de la valeur maximale
      /// </summary>
    public double? PercentRollUpperThird { get; set; }

      /// <summary>
      /// Le pourcentage des échantillons où le il y a un turbo
      /// </summary>
    public double? PercentTurbo { get; set; }

    //   /// <summary>
    //   /// Moyenne absolue de différences de vecteurs-vitesses
    //   /// </summary>
    // public double? AvgDiffVectorSpeed { get; set; }
  }
}

In [114]:
using System;
using System.Globalization;
using System.Net.Http;
using System.Net;
using System.Text.Json;
using GBX.NET;
using GBX.NET.Engines.Game;
using GBX.NET.Engines.Scene;
using GBX.NET.LZO;
using GBX.NET.ZLib;

#nullable enable

Gbx.LZO = new Lzo();
Gbx.ZLib = new ZLib();

public class TMXAPI
{
  public static HttpClient sharedClient = new()
  {
      BaseAddress = new Uri("https://tmnf.exchange"),
  };

  static TMXAPI() {
    sharedClient.DefaultRequestHeaders.UserAgent.ParseAdd("SchoolStatsProject/1.0 (+https://github.com/DarkMattrMaestro/StatsProjectTMNF)");
  }

  // private static async Task<TMXResponses.ParsedResponse<TMXResponses.MapResponse>?> getMapResponse(TMXInputs.MapInputsParameters paramaters)
  // {
  //   using HttpResponseMessage response = await sharedClient.GetAsync($"api/tracks?{paramaters}"); // Note: If "parameters" can be chosen by the user, add sanitization

  //   Console.WriteLine($"Get: api/tracks?{paramaters}");
    
  //   response.EnsureSuccessStatusCode();
    
  //   var jsonResponse = await response.Content.ReadAsStringAsync();

  //   TMXResponses.ParsedResponse<TMXResponses.MapResponse>? mapResponse = JsonSerializer.Deserialize<TMXResponses.ParsedResponse<TMXResponses.MapResponse>>(jsonResponse);

  //   return mapResponse;
  // }

  public static async Task<TMXResponses.ParsedResponse<TMXResponses.MapResponse>?> getMapResponseFromID(String[] fields, String id)
  {
    String relativeURL = $"api/tracks?fields={String.Join("%2C", fields)}&id={id}";
    using HttpResponseMessage response = await sharedClient.GetAsync(relativeURL); // Note: If "parameters" can be chosen by the user, add sanitization

    // Console.WriteLine(relativeURL); //
    
    response.EnsureSuccessStatusCode();
    
    var jsonResponse = await response.Content.ReadAsStringAsync();

    TMXResponses.ParsedResponse<TMXResponses.MapResponse>? mapResponse = JsonSerializer.Deserialize<TMXResponses.ParsedResponse<TMXResponses.MapResponse>>(jsonResponse);

    return mapResponse;
  }

  public static async Task<TMXResponses.ParsedResponse<TMXResponses.ReplayResponse>?> getTopReplaysResponseFromMapID(String[] fields, String trackId)
  {
    TMXInputs.ReplayInputsParameters inputsParameters = new TMXInputs.ReplayInputsParameters{ trackId = trackId, fields = fields, count = SampleSettings.minRecords};

    String relativeURL = $"api/replays?{inputsParameters}";
    using HttpResponseMessage response = await sharedClient.GetAsync(relativeURL); // Note: If "parameters" can be chosen by the user, add sanitization

    // Console.WriteLine(relativeURL); //
    
    response.EnsureSuccessStatusCode();
    
    var jsonResponse = await response.Content.ReadAsStringAsync();

    TMXResponses.ParsedResponse<TMXResponses.ReplayResponse>? replayResponse = JsonSerializer.Deserialize<TMXResponses.ParsedResponse<TMXResponses.ReplayResponse>>(jsonResponse);

    return replayResponse;
  }

  // Voir: https://github.com/BigBang1112/randomizer-tmf/blob/bae33563fe4116b22be5b099a51a5d311dd3366c/Src/RandomizerTMF.Logic/Services/MapDownloader.cs#L200
  private static Uri? GetRequestUriFromResponse(HttpResponseMessage response)
  {
      if (response.RequestMessage is null)
      {
          Console.Error.WriteLine("Response from the HEAD request does not contain information about the request message. This is odd...");
          return null;
      }

      if (response.RequestMessage.RequestUri is null)
      {
          Console.Error.WriteLine("Response from the HEAD request does not contain information about the request URI. This is odd...");
          return null;
      }

      return response.RequestMessage.RequestUri;
  }

  // Voir https://github.com/BigBang1112/randomizer-tmf/blob/bae33563fe4116b22be5b099a51a5d311dd3366c/Src/RandomizerTMF.Logic/Services/MapDownloader.cs#L217
  private static string? GetTrackIdFromUri(Uri uri)
  {
      var trackId = uri.Segments.LastOrDefault();

      if (trackId is null) { return null; }

      return trackId;
  }

  // Voir: https://github.com/BigBang1112/randomizer-tmf/blob/bae33563fe4116b22be5b099a51a5d311dd3366c/Src/RandomizerTMF.Logic/Services/MapDownloader.cs
  public static async Task<string?> getRandomMapID(TMXInputs.MapInputsParameters paramaters)
  {
    string? trackId = null;

    Uri? requestUri = null;
    
    bool retry = true;
    for (int doubleCheck = 1; doubleCheck < 3 && retry; doubleCheck++) {
      retry = false;

      using var response = await sharedClient.SendAsync(new HttpRequestMessage(HttpMethod.Head, $"trackrandom?{paramaters}")); // Note: If "parameters" can be chosen by the user, add sanitization

      if (response.StatusCode == HttpStatusCode.NotFound)
      {
        // I have written something incorrectly

        throw new Exception("No map meets the filter; the programmer has written erronous parameters");
      }

      if (response.StatusCode == HttpStatusCode.InternalServerError)
      {
        // The query is invalid or an unknown internal server error

        Console.WriteLine("\n\n" + new HttpRequestMessage(HttpMethod.Head, $"trackrandom?{paramaters}"));

        await Task.Delay(1000);

        if (doubleCheck < 2) {
          retry = true;
          continue;
        }

        throw new Exception("500 error");
      }
    
    response.EnsureSuccessStatusCode();
    
    requestUri = GetRequestUriFromResponse(response);
    }

    if (requestUri is null)
    {
        Console.Error.WriteLine("RequestUri of /trackrandom is null");
        return null;
    }

    trackId = GetTrackIdFromUri(requestUri);

    return trackId;
  }

  public static async Task<CGameCtnReplayRecord> getReplayFilestream(string replayId) {
    String relativeURL = $"recordgbx/{replayId}";

    using HttpResponseMessage response = await sharedClient.GetAsync(relativeURL);

    response.EnsureSuccessStatusCode();
    
    var gbxResponse = Gbx.Parse<CGameCtnReplayRecord>(response.Content.ReadAsStream());

    return gbxResponse;
  }

  /// <summary>
  /// CETTE MÉTHODE NE PROCURE PAS L'ÉTIQUETTE
  /// </summary>
  public static async Task<DataContainers.ReplayData> getReplayData(string replayId) {
    DataContainers.ReplayData replayData = new DataContainers.ReplayData();

    CGameCtnReplayRecord replay = await getReplayFilestream(replayId); // Terrible error management

    CGameCtnGhost ghost = replay.GetGhosts().First<CGameCtnGhost>();

    replayData.Duration = ghost.RaceTime!.Value.TotalSeconds;

    List<DataContainers.SampleData> samples = new List<DataContainers.SampleData>();

    // DataContainers.Vector3 displacement = new DataContainers.Vector3();
    foreach (CSceneVehicleCar.Sample sample in ghost.SampleData!.Samples) {

      DataContainers.SampleData sampleData = new DataContainers.SampleData();

      // Temps actuel de l'échantillon
      sampleData.Time = sample.Time.TotalSeconds;

      // Révolutions par minute
      sampleData.RPM = sample.RPM;

      // Pilotage
      sampleData.Steer = sample.Steer;
      
      // Vitesses vers l'avant et latérale
      sampleData.SpeedForward = sample.SpeedForward;
      sampleData.SpeedSideward = sample.SpeedSideward;

      // Vitesse angulaire
      sampleData.AngularVelocity!.X = sample.AngularVelocity.X;
      sampleData.AngularVelocity.Y = sample.AngularVelocity.Y;
      sampleData.AngularVelocity.Z = sample.AngularVelocity.Z;

      // Tangage, lacet, roulis
      GBX.NET.Quat quat = sample.Rotation;
      GBX.NET.Vec3 pitchYawRoll = quat.ToPitchYawRoll();
      sampleData.Pitch = pitchYawRoll.X * 180 / Math.PI;
      sampleData.Yaw = pitchYawRoll.Y * 180 / Math.PI;
      sampleData.Roll = pitchYawRoll.Z * 180 / Math.PI;

      // Position 3-dimensionnelle
      sampleData.Position!.X = sample.Position.X;
      sampleData.Position.Y = sample.Position.Y;
      sampleData.Position.Z = sample.Position.Z;

      // Présence d'un turbo
      sampleData.IsTurbo = sample.IsTurbo;
      
      // // Matériaux de contact des roues
      // sampleData.FRGroundContactMaterial = sample.FRGroundContactMaterial;
      // sampleData.FLGroundContactMaterial = sample.FLGroundContactMaterial;
      // sampleData.RRGroundContactMaterial = sample.RRGroundContactMaterial;
      // sampleData.RLGroundContactMaterial = sample.RLGroundContactMaterial;

      samples.Add(sampleData);
    }

    replayData.Samples = samples;

    return replayData;
  }
}


In [115]:
public class ProgressBar {
  static String liveIndicators = @"\|/-";
  int indicator = 0;

  String previousText = "";

  public void Redraw(int collectedReplays, int totalReplays, Tag tag, int collectedSamples, int totalSamples) {
    indicator = (indicator + 1) % liveIndicators.Length;

    String currentText = $"\x000D{liveIndicators[indicator]} {collectedReplays}/{totalReplays} [{new string('#', 10 * collectedReplays / totalReplays)}{new string('-', 10 * (totalReplays - collectedReplays) / totalReplays)}] ({100 * collectedReplays / totalReplays}%)";

    currentText += $"  -  {collectedSamples}/{totalSamples} ({tag.ToString()}) [{new string('#', 5 * (collectedSamples) / totalSamples)}{new string('-', 5 * (totalSamples - collectedSamples) / totalSamples)}] ({100 * (collectedSamples) / totalSamples}%)";

    previousText = $"{currentText}{new string(' ', Math.Max(previousText.TrimEnd(' ').Length - currentText.TrimEnd(' ').Length, 0))}";
    Console.Write(previousText);
  }

  public void RedrawIndicator() {
    indicator = (indicator + 1) % liveIndicators.Length;

    Console.Write($"\x000D{liveIndicators[indicator]}");
  }
}

## Sélection aléatoire stratifié de circuits et de _replays_

Pour sélectionner un circuit aléatoire, nous utilisons le sous-répertoire `/trackrandom` du site TMNF-X. Les `minRecords` _replays_ les plus rapides -- de joueurs différents -- sont ensuite cherchés. Si un nombre de _replays_ équivalent à `minRecords` sont trouvés, les identifiants du circuit et du record mondial sont enregistrés.

In [116]:
using CsvHelper;
using System.Globalization;
using System.IO;

public class ObjectConversion {
  public static DataContainers.FlatData FlattenReplayData(DataContainers.ReplayData replayData) {
    DataContainers.FlatData flatData = new DataContainers.FlatData();

    // Initializer les variables comptants

    double SumAbsDisplacementX = 0;
    double SumAbsDisplacementY = 0;
    double SumAbsDisplacementZ = 0;

    double SumRPM = 0;

    double SumSteerBias = 0;
    double SumAbsSteer = 0;

    double SumSpeedForward = 0;
    double SumAbsSpeedForward = 0;

    double SumSpeedSidewardBias = 0;
    double SumAbsSpeedSideward = 0;

    double SumSpeedSidewardOppSteer = 0;

    double CountPitchLowerThird = 0;
    double CountPitchMiddleThird = 0;
    double CountPitchUpperThird = 0;

    double CountRollLowerThird = 0;
    double CountRollMiddleThird = 0;
    double CountRollUpperThird = 0;

    double CountTurbo = 0;

    // double SumDiffVectorSpeed = 0;

    // Compter

    int numSamples = replayData.Samples.Count;
    List<DataContainers.SampleData> allSampleData = replayData.Samples;
    DataContainers.Vector3 lastPosition = null;
    // DataContainers.Vector3 lastVectorSpeed = null;
    foreach (DataContainers.SampleData sampleData in allSampleData) {
      if (lastPosition != null) { // Ignorer le cas d'une soustraction avec un échantillon qui précède le premier échantillon
        SumAbsDisplacementX += Math.Abs(sampleData.Position.X - lastPosition.X);
        SumAbsDisplacementY += Math.Abs(sampleData.Position.Y - lastPosition.Y);
        SumAbsDisplacementZ += Math.Abs(sampleData.Position.Z - lastPosition.Z);

        // double diffX = sampleData.AngularVelocity.X - lastVectorSpeed.X;
        // double diffY = sampleData.AngularVelocity.Y - lastVectorSpeed.Y;
        // double diffZ = sampleData.AngularVelocity.Z - lastVectorSpeed.Z;
        // SumDiffVectorSpeed += Math.Sqrt(diffX*diffX + diffY*diffY + diffZ*diffZ);
      }
      lastPosition = sampleData.Position;
      // lastVectorSpeed = sampleData.AngularVelocity;

      SumRPM += (double)sampleData.RPM;

      SumSteerBias += (double)sampleData.Steer;
      SumAbsSteer += Math.Abs((double)sampleData.Steer);

      SumSpeedForward += (double)sampleData.SpeedForward;
      SumAbsSpeedForward += Math.Abs((double)sampleData.SpeedForward);

      SumSpeedSidewardBias += (double)sampleData.SpeedSideward;
      SumAbsSpeedSideward += Math.Abs((double)sampleData.SpeedSideward);

      SumSpeedSidewardOppSteer += Math.Sign((double)sampleData.SpeedSideward) * Math.Sign((double)sampleData.Steer);

      if (Math.Abs((double)sampleData.Pitch) < 60) { CountPitchLowerThird++; }
      else if (Math.Abs((double)sampleData.Pitch) < 120) { CountPitchMiddleThird++; }
      else { CountPitchUpperThird++; }

      if (Math.Abs((double)sampleData.Roll) < 60) { CountRollLowerThird++; }
      else if (Math.Abs((double)sampleData.Roll) < 120) { CountRollMiddleThird++; }
      else { CountRollUpperThird++; }

      if (sampleData.IsTurbo ?? false) { CountTurbo++; }
    }

    // Assigner les valeurs calculées

    flatData.Tag = Enum.Parse<Tag>(replayData.Tag, true).ToString();

    flatData.AvgAbsDisplacementHorizontal = Math.Sqrt(SumAbsDisplacementX*SumAbsDisplacementX + SumAbsDisplacementZ*SumAbsDisplacementZ) / (numSamples - 1); // L'on soustrait un puisque le premier échantillon est sauté
    flatData.AvgAbsDisplacementY = SumAbsDisplacementY / (numSamples - 1); // L'on soustrait un puisque le premier échantillon est sauté

    flatData.AvgRPM = SumRPM / numSamples;

    flatData.AvgSteerBias = Math.Abs(SumSteerBias / numSamples);
    flatData.AvgAbsSteer = SumAbsSteer / numSamples;

    flatData.AvgSpeedForward = SumSpeedForward / numSamples;
    flatData.AvgAbsSpeedForward = SumAbsSpeedForward / numSamples;

    flatData.AvgSpeedSidewardBias = Math.Abs(SumSpeedSidewardBias / numSamples);
    flatData.AvgAbsSpeedSideward = SumAbsSpeedSideward / numSamples;

    flatData.AvgSpeedSidewardOppSteer = SumSpeedSidewardOppSteer / numSamples;

    flatData.PercentPitchLowerThird = CountPitchLowerThird / numSamples;
    flatData.PercentPitchMiddleThird = CountPitchMiddleThird / numSamples;
    flatData.PercentPitchUpperThird = CountPitchUpperThird / numSamples;

    flatData.PercentRollLowerThird = CountRollLowerThird / numSamples;
    flatData.PercentRollMiddleThird = CountRollMiddleThird / numSamples;
    flatData.PercentRollUpperThird = CountRollUpperThird / numSamples;

    flatData.PercentTurbo = CountTurbo / numSamples;

    // flatData.AvgDiffVectorSpeed = SumDiffVectorSpeed / (numSamples - 1); // L'on soustrait un puisque le premier échantillon est sauté

    return flatData;
  }

  public static List<DataContainers.FlatData> FlattenAllReplayData(List<DataContainers.ReplayData> allReplayData) {
    List<DataContainers.FlatData> allFlatData = new List<DataContainers.FlatData>();

    foreach (DataContainers.ReplayData replayData in allReplayData) {
      DataContainers.FlatData flatData = ObjectConversion.FlattenReplayData(replayData);

      // Ajouter le replay applati à la liste des replays applatis
      allFlatData.Add(flatData);
    }

    return allFlatData;
  }

  public static void AllFlatDataToCSV(List<DataContainers.FlatData> allFlatData, string fileName, bool append = false) {
    bool shouldAppend = append && File.Exists(fileName);

    using (StreamWriter writer = new StreamWriter(fileName, shouldAppend)) {
      using (CsvWriter csv = new CsvWriter(writer, CultureInfo.InvariantCulture)) {
        if (!shouldAppend) {
          csv.WriteHeader<DataContainers.FlatData>();
          csv.NextRecord();
        }
        
        foreach (DataContainers.FlatData flatData in allFlatData)
        {
            csv.WriteRecord(flatData);
            csv.NextRecord();
        }
      }
    }
  }

  public static void AllReplayDataFlattenAndCSV(List<DataContainers.ReplayData> allReplayData, string fileName, bool append = false) {
    // Flatten
    List<DataContainers.FlatData> allFlatData = ObjectConversion.FlattenAllReplayData(allReplayData);
    
    // To CSV
    ObjectConversion.AllFlatDataToCSV(allFlatData, fileName, append);
  }
}

In [117]:
using System.Text.Json;
using System.Text.Json.Serialization;
using System.IO;
using System;
using System.Threading;

public class RandomMap()
{
  // public static Dictionary<Tag, List<String[]>> chosenMaps = (Enum.GetValues<Tag>()).ToDictionary(key => key, value => new List<String[]>()); // Stockage des identifiants de circuits et replays. À utiliser seulement lors de l'écriture du code.

  public static List<DataContainers.ReplayData> allReplayData = new List<DataContainers.ReplayData>();

  public static async Task selectMapIds() {
    Console.WriteLine($"Échantillonage des étiquettes... ");

    int totalReplays = SampleSettings.sampleSizePerTag * Enum.GetValues<UsableTag>().Length;
    int collectedReplays = 0;

    ProgressBar progressBar = new ProgressBar();

    // Loop for every sample per tag
    for (int remaining = SampleSettings.sampleSizePerTag; remaining > 0; remaining--) {
      List<DataContainers.ReplayData> checkpointReplayData = new List<DataContainers.ReplayData>(); // This itteration's collected replay data

      // Setup for loops
      int tagIndex = 0;
      Tag[] usableTags = Array.ConvertAll(Enum.GetValues<UsableTag>(), value => (Tag)value);

      // Visuel
      progressBar.Redraw(collectedReplays, totalReplays, usableTags[0], 0, usableTags.Length);

      // Loop over every tag
      foreach (Tag tag in usableTags) {
        tagIndex++;

        // Choisir un circuit valide et replay
        string chosenId = null;
        int queryAttempts = 0;
        bool successfulQuery = false;
        while (queryAttempts < SampleSettings.hardMaximumAttemptsPerQuery && !successfulQuery) { // Hard attempts limit
          for (int softAttemptsRemaining = SampleSettings.softMaximumAttemptsPerQuery; softAttemptsRemaining > 0 && !successfulQuery; softAttemptsRemaining--) { // Soft attempts limit            
            queryAttempts++; // Increment query attempts counter

            // Select a map, and get its id
            chosenId = await TMXAPI.getRandomMapID(new TMXInputs.MapInputsParameters{tag = [tag], etag = Enum.GetValues<Tag>().Except([tag]).Take(10).ToArray()}); // TODO: Figure 500 error

            // Unsure map only has one assigned tag
            TMXResponses.ParsedResponse<TMXResponses.MapResponse> mapResponse = await TMXAPI.getMapResponseFromID(["Tags"], chosenId);
            if (mapResponse.Results[0].Tags.Length > 1) {
              progressBar.RedrawIndicator(); // Redraw indicator
              continue;
            }

            await Task.Delay(200); // Small rate limiter

            // Get replay JSON
            TMXResponses.ParsedResponse<TMXResponses.ReplayResponse> replaysResponse = await TMXAPI.getTopReplaysResponseFromMapID(["ReplayId","User.Name"], chosenId);

            // Vérifier que le nombre minimal de records, défini dans les paramètres, sont présents
            if(replaysResponse.Results.Length < SampleSettings.minRecords) {
              progressBar.RedrawIndicator(); // Redraw indicator
              continue;
            }

            // Primary checks have passed, the map is likely valid

            // Get replay data
            DataContainers.ReplayData replayData = null;
            try {
              replayData = await TMXAPI.getReplayData(replaysResponse.Results[0].ReplayId.ToString());

              // Ensure there is at least one sample.
              // Serves the double-use of ensuring .Samples is not null
              if (replayData.Samples.Count < 1) {
                continue;
              }
            } catch (Exception e) {
              Console.WriteLine(e.Message);
              await Task.Delay(20000);
              continue;
            }
            replayData.Tag = tag.ToString();

            // Add replay data to the list of all, and of the checkpoint, replay data
            allReplayData.Add(replayData);
            checkpointReplayData.Add(replayData);

            // Update completion indicators
            collectedReplays++;
            successfulQuery = true;

            progressBar.Redraw(collectedReplays, totalReplays, tag, tagIndex, usableTags.Length); // Redraw progress bar
          }

          // If soft attempt limit hit, wait a bit
          if (!successfulQuery) {
            Console.Write(" *Soft limit reached*");
            await Task.Delay(20000);
          }
        }
        // Si trop d'essaies d'échentillonnage ont été faillis, indiquer une erreur
        if (!successfulQuery) {
          Console.WriteLine($"Map querying failed at ({collectedReplays + 1})th attempted sample with tag ({tag})");
          throw new HttpRequestException($"Map querying failed at ({collectedReplays + 1})th attempted sample with tag ({tag})");
        }
      }

      // Save checkpoint of collected replay data as CSV
      ObjectConversion.AllReplayDataFlattenAndCSV(checkpointReplayData, GeneralSettings.combinedFlatReplayDataFileName, true);
      ObjectConversion.AllReplayDataFlattenAndCSV(checkpointReplayData, GeneralSettings.AppendIndividualDataFolderName(GeneralSettings.flatReplayDataFileName), true);

      // Save checkpoint of collected replay data as JSON
      RandomMap.AppendReplayDataToJSON(checkpointReplayData, GeneralSettings.combinedReplayDataFileName);
      RandomMap.AppendReplayDataToJSON(checkpointReplayData, GeneralSettings.AppendIndividualDataFolderName(GeneralSettings.replayDataFileName));
    }
  }

  public static async Task replayDataToFile() {
    if (allReplayData.Count < 1) {
      await RandomMap.selectMapIds();
    }

    // string json = JsonSerializer.Serialize(RandomMap.allReplayData);

    // File.WriteAllText(GeneralSettings.AppendIndividualDataFolderName(GeneralSettings.replayDataFileName), json);
  }

  /////////////////// Added ____

  public static void AppendReplayDataToJSON(List<DataContainers.ReplayData> data, string fileName) {
    bool shouldAppend = File.Exists(fileName);

    if (shouldAppend) {
      String json = File.ReadAllText(fileName);
      List<DataContainers.ReplayData> allReadReplayData = JsonSerializer.Deserialize<List<DataContainers.ReplayData>>(json);

      allReadReplayData.AddRange(data);

      string newJson = JsonSerializer.Serialize(allReadReplayData);
      File.WriteAllText(fileName, newJson);
    } else {
      string newJson = JsonSerializer.Serialize(data);
      File.WriteAllText(fileName, newJson);
    }
  }

  /////////////////// Added UpUpUpUpUpUp
}


In [118]:
using System.Text.Json;
using System.Text.Json.Serialization;
using System.IO;
using System;
using System.Threading;
using System.Globalization;

await RandomMap.replayDataToFile();

// if (!File.Exists(GeneralSettings.replayDataFileName)) {
//   await RandomMap.replayDataToFile();
// }

String json = File.ReadAllText(GeneralSettings.AppendIndividualDataFolderName(GeneralSettings.replayDataFileName));
List<DataContainers.ReplayData> allReadReplayData = JsonSerializer.Deserialize<List<DataContainers.ReplayData>>(json);

// allReadReplayData.Display();

// ObjectConversion.AllReplayDataFlattenAndCSV(allReadReplayData, GeneralSettings.AppendIndividualDataFolderName(GeneralSettings.flatReplayDataFileName), false); // Done elsewhere in the program

// Afficher le tableau des données

// allFlatData.DisplayTable();

Échantillonage des étiquettes... 
- 8/8 [##########] (100%)  -  8/8 (Grass) [#####] (100%)   